In [6]:
from ultralytics import YOLO

trained = YOLO('Human-Detection.pt')

In [7]:
import time
import requests

In [8]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
from ultralytics import YOLO
import cv2

# Open the webcam 
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

width = cap.get(3)  # float `width`
height = cap.get(4)  # float `height`

track_times_way_1 = {}
track_times_way_2 = {}

area_1 = ((0,0), (200,int(height)))
area_2 = ((int(width)-200,0), (int(width),int(height)))

while True:

    # Capture frame-by-frame
    ret, frame = cap.read()

    if not ret:
        print("Error: Failed to capture image.")
        break

    # Run YOLO inference on the frame
    results = trained.track(frame, persist=True, verbose=False, conf=0.2, classes=[0])

    current_tracks = set()

    for result in results:  # Results as [x1, y1, x2, y2, confidence, class]
        for i in range(len(result.boxes.cls)):
            x1, y1, x2, y2 = result.boxes.xyxy[i]
            conf = result.boxes.conf[i].item()
            try:
                if(result.boxes.id[i]):
                    id = result.boxes.id[i].item()
            except:
                pass
            x1 = x1.item()
            y1 = y1.item()
            x2 = x2.item()
            y2 = y2.item()

            mid_x = (x1 + x2) / 2  # Middle of the bounding box
            mid_y = (y1 + y2) / 2
            
            if area_1[0][0] <= mid_x <= area_1[1][0] and area_1[0][1] <= mid_y <= area_1[1][1]:  # Check if in green zone
                if id not in track_times_way_1:
                    track_times_way_1[id] = time.time()  # Record entry time
                if id in track_times_way_2:
                    elapsed_time = time.time() - track_times_way_2[id]
                    print(f"Time from red to green for ID {id}: {elapsed_time:.2f} seconds")
                    response = requests.post(os.getenv('URL'), json={"args": {"cross_duration": round(elapsed_time,2)}})
                    print(response)
                    del track_times_way_2[id]  # Remove tracking once completed
            if area_2[0][0] <= mid_x <= area_2[1][0] and area_2[0][1] <= mid_y<= area_2[1][1]:  # Check if in red zone
                if id not in track_times_way_2:
                    track_times_way_2[id] = time.time()  # Record entry time
                if id in track_times_way_1:
                    elapsed_time = time.time() - track_times_way_1[id]
                    response = requests.post(os.getenv('URL'), json={"args": {"cross_duration": round(elapsed_time,2)}})
                    print(response)
                    del track_times_way_1[id]  # Remove tracking once completed

            current_tracks.add(id)

    # Cleanup track_times to remove old tracks no longer in view
    track_ids_to_remove_1 = [tid for tid in track_times_way_1 if tid not in current_tracks]
    for tid in track_ids_to_remove_1:
        del track_times_way_1[tid]
    
    track_ids_to_remove_2 = [tid for tid in track_times_way_2 if tid not in current_tracks]
    for tid in track_ids_to_remove_2:
        del track_times_way_2[tid]


    cv2.rectangle(frame, area_1[0], area_1[1], (0, 255, 0), 2)
    cv2.rectangle(frame, area_2[0], area_2[1], (0, 0, 255), 2)

    # Visualize the results on the frame
    annotated_frame = results[0].plot()  # YOLOv8 has built-in plotting for results

    # Display the resulting frame with annotations
    cv2.imshow('YOLO Webcam', annotated_frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# When everything is done, release the capture and close the window
cap.release()
cv2.destroyAllWindows()




2024-09-14 18:47:04.213 Python[30685:1435315] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


Time from red to green for ID 2.0: 4.93 seconds


AttributeError: 'float' object has no attribute 'round'